In [1]:
import pandas as pd
import numpy as np
import os
import glob
import random
import csv
from tqdm import tqdm


from datetime import datetime as dt
from datetime import timedelta
from sklearn.model_selection import train_test_split

In [2]:
final_dir = '/data_data/session_length/nishanth01/data/final/'

In [4]:
def val_int(val):
    if (pd.isnull(val) or val == 'NA'):
        return 0
    else:
        return val

def val_str(val):
    if (pd.isnull(val) or val == 'NA'):
        return ''
    else:
        val = val.encode('ascii', 'ignore')
        return val.strip('"').strip("'").strip().lower()
    
def val_genre(val):
    if (pd.isnull(val) or val == 'NA'):
        return ''
    else:
        return val.strip('"').strip("'").strip().lower()    
    
    
def parse_date(date):
    if (pd.isnull(date) or date == 'NA'):
        return ''
    else:
        return pd.to_datetime(date)    


def get_users(test=0.2,val=0.1):
    columns = ['user_id']
    users = pd.read_csv('/data_data/session_length/nishanth01/data/unique_users.txt',names=columns)
    _,test_df = train_test_split(users, test_size=test+val)
    test_users,val_users = train_test_split(test_df, test_size=(val)/(test+val))
    return users,test_users,val_users


def get_user_details(user_id):
    file_name = '/data_data/session_length/nishanth01/data/users/{0}/*.csv'.format(user_id) 
    columns = ['user_id','timestamp','artist_name',
               'track_name','gender','age','country',
               'registered','duration','genre']
    
    complete_files = glob.glob(file_name)
    user_data = pd.concat((pd.read_csv(f,names=columns,sep='\t') for f in complete_files))
    return user_data

def get_seconds(in_ms):
    return 0.001*in_ms


def get_times_played(time,duration):
    if(duration != 0):
        return time / duration
    else:
        return 0

    
def get_time_difference(t1,t2):
    time1 = dt.strptime(t1,'%Y-%m-%dt%H:%M:%Sz')
    time2 = dt.strptime(t2,'%Y-%m-%dt%H:%M:%Sz')
    return (time2-time1).total_seconds()


def max_session_window():
    return timedelta(minutes=30).total_seconds() 
    
    
def process_prev_row(row,time,session_id,session_length):
    data = []
    new_session = False
    prev_time = row[1]['timestamp']
    curr_time = time
    times_played = 0
    
    try:
        track_duration = get_seconds(float(row[1]['duration']))  
    except Exception:
        track_duration = 0

    try:
        diff = get_time_difference(prev_time,curr_time)
        
        if(track_duration > 0):
            if(diff <= track_duration):#same session
                session_length = session_length + diff
                times_played = get_times_played(diff,track_duration)
            else:
                if((diff-track_duration) > max_session_window()):#next is new session
                    session_length = session_length + track_duration
                    times_played = get_times_played(track_duration,track_duration)
                    new_session = True
                else:    
                    session_length = session_length + diff    
                    times_played = get_times_played(diff,track_duration)
        else:    
            if(diff > max_session_window()):#next is new session
                new_session = True
            else:    
                session_length = session_length + diff    

        data.append(row[1]['user_id'])#user_id
        data.append(row[1]['timestamp'])#timestamp
        data.append(session_id)#session_id
        data.append(track_duration)#track_duration
        data.append(times_played)#times_played        
        data.append(row[1]['gender'])#gender
        data.append(row[1]['age'])#age
        data.append(row[1]['country'])#country
        data.append(row[1]['registered'])#registered
        data.append(row[1]['artist_name'])#artist_name
        data.append(row[1]['track_name'])#track_name
        data.append(session_length)#session_length
        data.append(row[1]['genre'])#genre
        
    except Exception as e:
        #print('Exception 2:: Skipping')
        new_session = True
#         raise Exception('Inner error..!')
        
    return new_session,session_length,data

def create_user(user_id,test,val):
    user = get_user_details(user_id)
    user = user.sort_values(by=['timestamp'])
    total_count = len(user.index)
    test_index = int(total_count*random.uniform(0.5, 1))
    
    test_file = ''
    train_file = final_dir+'train/{0}.csv'.format(user_id)
    if(test):
        test_file = final_dir+'test/{0}.csv'.format(user_id)
    else:
        test_file = final_dir+'validate/{0}.csv'.format(user_id)    

        
    session_length = 0
    session_id = 1
    curr_time = ''
    prev_time = ''
    new_session = False
    i = 0
    delete = False
    if(not test and not  val):
        delete = True
    
    with open(train_file,'w+') as train_out,open(test_file,'w') as test_out:
        train_out = csv.writer(train_out,quoting=csv.QUOTE_NONNUMERIC,delimiter='\t')
        test_out = csv.writer(test_out,quoting=csv.QUOTE_NONNUMERIC,delimiter='\t')
        for row in tqdm(user.iterrows(),total=total_count):
            try:
                data = []
                if(i == 0):#first time
                    session_id = 1
                    prev_row = row
                else:    
                    if(prev_row):
                        new_session,session_length,data = process_prev_row(prev_row,
                                                                           row[1]['timestamp'],
                                                                           session_id,
                                                                           session_length)
                        prev_row = row
                    else:
                        raise Exception('Unhandled error..!')

                    if(new_session):
                        session_id = session_id + 1
                        session_length = 0

                if(data):
                    if(test and i > test_index):
                        test_out.writerow(data)
                    elif(val and i > test_index):
                        test_out.writerow(data)
                    else:
                        train_out.writerow(data)        

            except Exception as e:
                print('EXCEPTION(1): Skipping...',e)
                pass

            i = i+1        
    if(delete):
        try:
            os.remove(test_file) 
        except OSError:
            pass
        
    print('COMPLETED: {0}'.format(user_id))

In [8]:
def process():
    print('Starting..')
    failed = []
    try:
        i = 0
        users,test_users,val_users = get_users()
        for row in users.iterrows():
            try:
                test = False
                val = False
                user_id = row[1]['user_id']
                if(len(test_users.loc[test_users['user_id'] == user_id].index)):
                    test = True
                elif (len(val_users.loc[val_users['user_id'] == user_id].index)):
                    val = True
                create_user(user_id,test,val)
            except Exception as e:
                failed.append(user_id)
                pass
            i += 1
#             if(i == 2):
#                 break
    except Exception as e:
        print('EXCEPTION 0 :::',e)
    finally:
        print('FAILED Users: ',failed)
        
    print('COMPLETE!')

In [9]:
!rm -rf /data_data/session_length/nishanth01/data/final/train/*
!rm -rf /data_data/session_length/nishanth01/data/final/test/*
!rm -rf /data_data/session_length/nishanth01/data/final/validate/*

In [10]:
process()

Starting..


100%|██████████| 16685/16685 [00:06<00:00, 2626.96it/s]


COMPLETED: user_000001


100%|██████████| 57512/57512 [00:19<00:00, 2985.17it/s]


COMPLETED: user_000002


...

COMPLETED: user_000998


100%|██████████| 31415/31415 [00:07<00:00, 4308.12it/s]


COMPLETED: user_000999


100%|██████████| 18373/18373 [00:04<00:00, 4316.85it/s]

COMPLETED: user_001000
('FAILED Users: ', ['user_000028', 'user_000029', 'user_000031', 'user_000033', 'user_000040', 'user_000041', 'user_000053', 'user_000060', 'user_000062', 'user_000069', 'user_000074', 'user_000089', 'user_000091', 'user_000107', 'user_000112', 'user_000121', 'user_000122', 'user_000125', 'user_000135', 'user_000138', 'user_000142', 'user_000158', 'user_000162', 'user_000174', 'user_000183', 'user_000210', 'user_000219', 'user_000237', 'user_000249', 'user_000274', 'user_000281', 'user_000296', 'user_000341', 'user_000359', 'user_000362', 'user_000366', 'user_000371', 'user_000397', 'user_000423', 'user_000425', 'user_000427', 'user_000439', 'user_000442', 'user_000468', 'user_000491', 'user_000504', 'user_000577', 'user_000585', 'user_000592', 'user_000595', 'user_000606', 'user_000607', 'user_000611', 'user_000661', 'user_000670', 'user_000681', 'user_000685', 'user_000706', 'user_000710', 'user_000730', 'user_000740', 'user_000750', 'user_000751', 'user_000756

In [11]:
!head /data_data/session_length/nishanth01/data/final/train/user_000001.csv